# Create train_data.csv

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
path_cosmenet = Path("/home/music/Desktop/measure_model/data/cosmenet_test")

In [3]:
df = []
for root, directories, files in os.walk(path_cosmenet):
    for file in files:
        classes = Path(root).name
        df.append([file, classes])

In [48]:
df_pd = pd.DataFrame(df, columns=['image_name', 'id_product'])

In [52]:
df_pd.to_csv(path_cosmenet / 'train_data.csv', index=False)

# Create Dataset

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

import pandas as pd

from PIL import Image
from pathlib import Path
import random

In [2]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
IMAGE_SIZE = 224
BATCH_SIZE = 4
DEVICE = get_default_device()
LEARNING_RATE = 0.00002
EPOCHS = 40

In [3]:
class CosmenetDataset_Triplet():
    def __init__(self, df: pd, path: Path, train=True, transform=None):
        self.data_csv = df
        self.is_train = train
        self.transform = transform
        self.path = path
        if self.is_train:
            self.images = df.iloc[:, 0].values
            self.labels = df.iloc[:, 1].values
            self.index = df.index.values 
    
    def full_path(self, label,  image_name):
        return self.path / str(label) / image_name
    
    def get_caompare_img(self, item, anchor_label, compare_type):
        if compare_type == "pos":
            compare_list = self.index[self.index!=item][self.labels[self.index!=item]==anchor_label]
        elif compare_type == "neg":
            compare_list = self.index[self.index!=item][self.labels[self.index!=item]!=anchor_label]
        else:
            raise ValueError("compare_type must be pos or neg")
        compare_item = random.choice(compare_list)
        compare_image_name = self.images[compare_item]
        compare_image_path = self.full_path(self.labels[compare_item], compare_image_name)
        compare_img = Image.open(compare_image_path).convert('RGB')
        return compare_img
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, item):
        anchor_label = self.labels[item]
        anchor_image_name = self.images[item]
        anchor_image_path = self.full_path(self.labels[item], anchor_image_name)
        anchor_img = Image.open(anchor_image_path).convert('RGB')
        if self.is_train:
            positive_img = self.get_caompare_img(item, anchor_label, "pos")
            negative_img = self.get_caompare_img(item, anchor_label, "neg")
            if self.transform!=None:
                anchor_img = (self.transform(anchor_img)*255).int()
                positive_img = (self.transform(positive_img)*255).int()
                negative_img = (self.transform(negative_img)*255).int()
        return anchor_img, positive_img, negative_img

In [4]:
# cosmenet test
train_data_path = Path("/app/nfs_clientshare/mew/project/Similarity_model/data/cosmenet_test")
train_data= pd.read_csv(train_data_path / 'train_data.csv') # [imag_path, label]
def get_train_dataset(IMAGE_SIZE):
    trans = transforms.Compose([transforms.ToTensor(),transforms.Resize((IMAGE_SIZE,IMAGE_SIZE), antialias=False)])
    train_dataset = CosmenetDataset_Triplet(train_data, path=train_data_path, train=True, transform=trans)
    return train_dataset

In [4]:
# real data
train_data_path = Path("/app/nfs_clientshare/Datasets/Cosmenet_product_20231018/datas")
train_data = pd.read_csv('/app/nfs_clientshare/Datasets/Cosmenet_product_20231018/datas_20231018.csv')
df_group = train_data.groupby('labels',sort=False).count()
filter_count = df_group[(df_group.file_names >= 20).values & (df_group.file_names < 40).values].index
train_data = train_data[train_data["labels"].isin(filter_count)].reset_index(drop=True)
def get_train_dataset(IMAGE_SIZE):
    trans = transforms.Compose([transforms.ToTensor(),transforms.Resize((IMAGE_SIZE,IMAGE_SIZE), antialias=False)])
    train_dataset = CosmenetDataset_Triplet(train_data, path=train_data_path, train=True, transform=trans)
    return train_dataset

In [5]:
train_dataset = get_train_dataset(IMAGE_SIZE)
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

# Preprocessing

In [6]:
import torch.nn as nn

In [8]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    def cosine(self, x1, x2):
        return self.cos(x1, x2)
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.cosine(anchor, positive)
        distance_negative = self.cosine(anchor, negative)
        losses = torch.relu(-distance_positive + distance_negative + self.margin)
        return losses.mean()

In [19]:
from transformers import ViTImageProcessor, ViTModel
vit_gg = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
vit_gg.eval().to(DEVICE)
processor_vit_gg = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
Optimizer = torch.optim.Adam(vit_gg.parameters(),lr = LEARNING_RATE)
criterion = nn.TripletMarginLoss(margin=1.0, p=2, eps=1e-8)
# criterion = TripletLoss()

In [20]:
# LAST_LAYER = 199
LAST_LAYER = 198
for n, (layer, param) in enumerate(vit_gg.named_parameters()):
    if n >= LAST_LAYER:
        print(layer)
        param.requires_grad = False
    else:
        param.requires_grad = True

pooler.dense.weight
pooler.dense.bias


# Validate

In [9]:
from script.tool import *

In [10]:
n_cv = 5
# path_dataset = '/home/music/Desktop/measure_model/data/product'
path_dataset = Path("/app/nfs_clientshare/Datasets/Cosmenet_products_15000/raw_data")
device = torch.device(DEVICE)
df = scan_directory(path_dataset)
df_pd, index_less_than_n, index_greater_than_or_equal_to_n = filter_data(df, minimum_data_class=n_cv)
y_label = df_pd['classes_labeled'].values

amount of all image : 15524
amount of image that less than 5 in that class : 116
amount of image that more than 5 in that class : 15408


In [16]:
def convert_feature_transformer(model, processor, layer, row=False, device='cuda:0'):
    model.eval().to(device)
    X_trans = []
    first = True
    for img_path in tqdm(df_pd['path_img'], desc="Extract"):
        img = Image.open(img_path).convert('RGB')
        inputs = processor(images=img, return_tensors="pt").to(device)
        outputs = model(**inputs)
        if type(row) == bool and row==False:
            output = outputs[layer]
        else:
            output = outputs[layer][:, row]
        output = output.flatten().unsqueeze(0)
        output = standardize_feature(output).to('cpu').detach().numpy()
        if first:
            X_trans = output
            first = False
        else:
            X_trans = np.concatenate((X_trans, output))
    
    df_x = pd.DataFrame(X_trans)
    df_y = pd.DataFrame(df_pd['classes'], columns=['classes'])
    data = pd.concat([df_x, df_y], axis=1)
    return data

In [17]:
def validate(data, n_cv=5):
    x_gg, y_gg = data.iloc[:, :-1], data.iloc[:, -1]
    X = x_gg
    y = y_label
    y_gg_un = y_gg
    index_filter=(index_greater_than_or_equal_to_n, index_less_than_n)

    first = True
    result_in_n = []
    skf = StratifiedKFold(n_splits=n_cv)
    if index_filter != False:
        index_greater_filtered, index_less_filtered = index_filter
        X_less = X[index_less_filtered]
        y_less = y[index_less_filtered]
        y_gg_un_less = y_gg_un[index_less_filtered]
        X = X[index_greater_filtered]
        y = y[index_greater_filtered]
        y_gg_un = y_gg_un[index_greater_filtered]
                
    index_df_split = skf.split(X, y)

    for train_index, test_index in tqdm(index_df_split,  desc="Validate"):
        x_train = np.array(X)[train_index]
        y_train = np.array(y)[train_index]
        y_gg_un_train = np.array(y_gg_un)[train_index]
        x_test = np.array(X)[test_index]
        y_test = np.array(y)[test_index]
        y_gg_un_test = np.array(y_gg_un)[test_index]
        
        if index_filter != False:
            x_train = np.concatenate((x_train, X_less))
            y_train = np.concatenate((y_train, y_less))
            y_gg_train = np.concatenate((y_gg_un_train, y_gg_un_less))

        dot_product = np.dot(x_test,x_train.T)              # (x_test , x_train)
        norm_test = norm(x_test, axis=1).reshape(-1, 1)     # (x_test, 1)
        norm_train = norm(x_train, axis=1).reshape(1, -1)   # (1, x_train)
        res = dot_product/(norm_test*norm_train)            # res = (x_test , x_train), norm_test*norm_train = (x_test , x_train)
        
        f = True
        rank_top_n = []
        ranking = np.argsort(res, axis=1)
        y_ranking = np.repeat(y_gg_train.reshape(1, -1), repeats=ranking.shape[0], axis=0)
        result_ranking = np.take_along_axis(y_ranking, ranking, axis=1)[:, ::-1]
        for row in result_ranking:
            indexes = np.unique(row, return_index=True)
            res_row = row[sorted(indexes[1])][:5].reshape(1, -1)
            if f:
                f = False
                rank_top_n = res_row
            else:
                rank_top_n = np.concatenate((rank_top_n, res_row))
        
        if first:
            first = False
            result_in_n = [rank_top_n]
        else:
            result_in_n.append(rank_top_n)
        
    result_avg = sum((y_gg_un_test.reshape(-1, 1) == result_in_n[-1]).any(axis=1))/result_in_n[-1].shape[0]
    return result_avg

# Training

In [13]:
from tqdm.notebook import tqdm
import numpy as np

In [21]:
# triple loss
best_performance = 0
for epoch in tqdm(range(EPOCHS), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(train_dl, desc="Training", leave=False)):
        anchor_img_pre = processor_vit_gg(images=anchor_img, return_tensors="pt").to(DEVICE)
        positive_img_pre = processor_vit_gg(images=positive_img, return_tensors="pt").to(DEVICE)
        negative_img_pre = processor_vit_gg(images=negative_img, return_tensors="pt").to(DEVICE)
        
        anchor_out = vit_gg(**anchor_img_pre).last_hidden_state[:, 0]
        positive_out = vit_gg(**positive_img_pre).last_hidden_state[:, 0]
        negative_out = vit_gg(**negative_img_pre).last_hidden_state[:, 0]
        
        loss = criterion(anchor_out, positive_out, negative_out)
        
        Optimizer.zero_grad()
        loss.backward()
        Optimizer.step()
        running_loss.append(loss.cpu().detach().numpy())
    
    x_trans = convert_feature_transformer(vit_gg, processor_vit_gg, layer="last_hidden_state", row=0, device=DEVICE)
    validation_loss = validate(x_trans)
    if validation_loss > best_performance:
        best_performance = validation_loss  # Update the best performance
        best_epoch = epoch
        sco = str(round(best_performance, 5)).split(".")
        path_trained = "weights/vit_gg_lr2e-05_eu_" + str(epoch+1) + "ep_" + sco[0] + "_" + sco[1] + "acc"
        vit_gg.save_pretrained(path_trained, from_pt=True)
        processor_vit_gg.save_pretrained(path_trained, from_pt=True)
        print("Best performance: {:.4f} at epoch {}".format(best_performance, best_epoch+1))
    
    if epoch%10 == 9:
        path_trained = "weights/vit_gg_lr2e-05_eu_" + str(epoch+1) + "ep"
        vit_gg.save_pretrained(path_trained, from_pt=True)
        processor_vit_gg.save_pretrained(path_trained, from_pt=True)
        print("Save model at epoch {}".format(epoch+1))

    print("Epoch: {}/{} — Loss: {:.4f} — acc : {:.4f}".format(epoch+1, EPOCHS, np.mean(running_loss), validation_loss))

Epochs:   0%|          | 0/40 [00:00<?, ?it/s]

Training:   0%|          | 0/678 [00:00<?, ?it/s]

Extract:   0%|          | 0/15524 [00:00<?, ?it/s]

Validate: 0it [00:00, ?it/s]

Best performance: 0.9448 at epoch 1
Epoch: 1/40 — Loss: 0.1041 — acc : 0.9448


...

Epoch: 39/40 — Loss: 0.0031 — acc : 0.9221


Training:   0%|          | 0/678 [00:00<?, ?it/s]

Extract:   0%|          | 0/15524 [00:00<?, ?it/s]

Validate: 0it [00:00, ?it/s]

Save model at epoch 40
Epoch: 40/40 — Loss: 0.0018 — acc : 0.9202


In [12]:
path_trained = "/home/music/Desktop/measure_model/weights/vit_gg_lr2e-05_eu_40ep"
vit_gg.save_pretrained(path_trained, from_pt=True)
processor_vit_gg.save_pretrained(path_trained, from_pt=True)

['/home/music/Desktop/measure_model/weights/vit_gg_lr2e-05_eu_40ep/preprocessor_config.json']

In [8]:
from transformers import ViTImageProcessor, ViTModel
path_trained = "/app/nfs_clientshare/mew/project/Similarity_model/weights/vit_gg_lr2e-05_pooler"
vit_gg = ViTModel.from_pretrained(path_trained)
vit_gg.eval().to(DEVICE)
processor_vit_gg = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
Optimizer = torch.optim.Adam(vit_gg.parameters(),lr = LEARNING_RATE)
criterion = nn.TripletMarginLoss(margin=1.0, p=2, eps=1e-8)

2023-10-19 13:25:54.818862: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-19 13:25:55.398070: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-19 13:25:55.398141: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-19 13:25:55.400202: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-19 13:25:55.632841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A